# 计算机视觉与图像处理实践课程设计自选项目：交通标志识别

## 一、小组成员
组长：杨蕊欣  任务系数：1.10  
负责查找资料与撰写文档  
成员：龙子腾  任务系数：1.00  
负责代码实现与调试

## 二、项目环境  
系统：Windows11 24H2  
环境：Anaconda 2.6.0 + Jupyter NoteBook 7.0.8  
框架：Pytorch 2.5.1 + CUDA 12.4

## 三、数据集信息  
* 本项目使用的数据集为德国交通标志基准测试(German Traffic Sign Recognition Benchmark,GTSRB),它拥有43个类别及50000多张图片，适用于单图像、多分类的问题。
* 数据集来源为Kaggle，下载地址：https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

## 四、项目实现

### 1.引入必要的库

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
from skimage import io, exposure, transform
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import imutils
import random
import cv2
import os
from torch.utils.data import Dataset
from torchviz import make_dot,make_dot_from_trace
import warnings

### 2.定义模型

定义一个模型，它接收的参数分别为`width`(宽度),`height`(高度),`depth`(深度),`classes`(类别数)

模型的总体结构为卷积层(`conv`)-全连接层(`fc`)-最大池化层(`pool`)，在每两层之间加入批量归一化层用于加速训练过程并提高模型稳定性。此外，应用Dropout以减少过拟合

模型的前向传播路径为：  
输入图像x首先通过第一个卷积层和批量归一化，然后应用ReLU激活函数和最大池化。  
接着，图像通过第二和第三个卷积层（每个后面都有ReLU激活），然后再次池化。  
图像继续通过第四和第五个卷积层（同样有ReLU激活），之后再次池化。  
经过所有卷积和池化层后，使用`torch.flatten`将特征图展平为一维向量。  
向量通过两个全连接层（每个后面都有ReLU激活和Dropout），最后通过输出层。  
输出层的结果通过`F.log_softmax`进行对数软化最大处理，用于多分类任务。  

In [2]:
class TrafficSignNet(nn.Module):
    def __init__(self, width, height, depth, classes):
        super(TrafficSignNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=depth, out_channels = 8, 
                               kernel_size = 5, padding = 2)
        self.bn1 = nn.BatchNorm2d(8)
        
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, 
                               kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, 
                               kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32,
                               kernel_size=3, padding = 1)
        self.bn4 = nn.BatchNorm2d(32)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32,
                               kernel_size=3, padding = 1)
        self.bn5 = nn.BatchNorm2d(32)

        self.fc1 = nn.Linear(32 * (height // 8) * (width // 8), 128)
        self.bn6 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128)
        self.bn7 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, classes)

        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = torch.flatten(x, 1)

        x = F.relu(self.bn6(self.fc1(x)))
        x = self.dropout(x)

        x = F.relu(self.bn7(self.fc2(x)))
        x = self.dropout(x)

        x = self.fc3(x)

        return F.log_softmax(x, dim=1)


### 3.定义数据集加载器

定义一个数据集加载器，它接收的参数为`base_path`(图像文件的根目录路径)与`csv_path`(包含图像路径和标签信息的 CSV 文件路径)

#### (1)数据读取  
打开csv文件，跳过文件的第一行，并对剩余行进行随机打乱，以增加数据集的多样性，随后遍历每一行，解析出标签和图像路径，并使用`os.path.join`将`base_path`和`image_path`合并成完整的图像文件路径。

#### (2)数据处理
使用`io.imread`方法读取图像文件，随后对图像预处理，将其裁剪为32×32的大小并应用自适应直方图均衡化。将处理后的图像和标签分别添加到`self.data`和`self.labels`列表中。

#### (3)数据转换
将`self.data`转换为NumPy数组，并将数据类型设置为float32，然后归一化到 [0, 1] 范围。  
同理，将`self.labels`转换为 NumPy 数组。

定义一个方法，获取数据集的长度

定义一个方法，根据索引`idx`从数据集中获取单个图像和标签,将图像从 NumPy 数组转换为PyTorch张量，并调整通道顺序以符合PyTorch的输入格式。并将标签转换为 PyTorch 长整型张量。随后返回图像和标签的张量。

In [3]:
class TrafficDataLoader(Dataset):
    def __init__(self, base_path, csv_path):
        self.data = []
        self.labels = []
        with open(csv_path, 'r') as f:
            rows = f.read().strip().split("\n")[1:]#跳过文件的第一行
            random.shuffle(rows)#对剩余行进行随机打乱
            for row in rows:#遍历每一行，解析出标签和图像路径
                label, image_path = row.strip().split(",")[-2:]
                image_path = os.path.join(base_path, image_path)#使用 os.path.join 将 base_path 和 image_path 合并成完整的图像文件路径。
                image = io.imread(image_path)#使用 io.imread读取图像文件
                image = transform.resize(image, (32, 32))
                image = exposure.equalize_adapthist(image, clip_limit=0.1)#应用自适应直方图均衡化
                self.data.append(image)
                self.labels.append(int(label))

        self.data = np.array(self.data, dtype="float32") / 255.0
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = torch.tensor(self.data[idx].transpose((2,0,1)), dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label


定义数据集的路径以及模型训练完成后的保存路径

In [4]:
args = {"dataset": "archive1/archive1/",
        "model":"output/trafficsignnet.pth"}

* 使用`os.path.join`函数来构建训练集和测试集CSV文件的完整路径  
* 使用上述数据加载器找到并处理数据
* 将数据分成若干组，每组六十四个数据，并将数据的顺序打乱。

In [5]:
train_path = os.path.join(args["dataset"], "Train.csv")
test_path = os.path.join(args["dataset"], "Test.csv")

train_dataset = TrafficDataLoader(args["dataset"], train_path)
test_dataset = TrafficDataLoader(args["dataset"], test_path)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle=False)

model = TrafficSignNet(width=32, height=32, 
                       depth = 3, classes = len(np.unique(train_dataset.labels)))

将模型移动到GPU进行训练

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

TrafficSignNet(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (bn6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2

#### (5)损失函数与优化器  
* 损失函数选择交叉熵损失函数（Cross Entropy Loss）,这是分类问题常用的损失函数
* 优化器选用Adam优化器  
* `model.parameters()`是一个生成器，它产生了模型中所有可训练的参数（通常是权重和偏置）。这些参数是优化器需要调整以减少损失函数的值的目标。  
* 定义学习率lr=0.001，有助于模型更稳定地收敛。

In [7]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

### 4.模型训练 

#### (1).定义训练函数

首先设置模型为训练模式，并初始化两个变量来跟踪在整个训练周期中正确预测的样本数和累积损失。使用`enumerate`来迭代`train_loader`，输出批次的索引和包含数据及其对应标签的元组。随后数据和标签都被移至GPU上。 

执行训练循环
* optimizer.zero_grad() 清空之前的梯度。
* model(data) 执行前向传播，生成预测输出。
* criterion(output, target) 计算损失。
* loss.backward() 执行反向传播，计算梯度。
* optimizer.step() 更新模型参数。

更新累积损失，并通过比较预测和真实标签来计算正确预测的样本数，每处理 100 个批次，打印一次当前的训练进度和损失。

In [8]:
def train(epoch):
    model.train()
    correct_train = 0
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pred = output.argmax(dim=1)
        correct_train += pred.eq(target).sum().item()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}")

#### (2)定义测试函数  
先把模型设置为评估模式，并初始化两个变量:`test_loss`用于累加测试损失，`correct`用于计数正确预测的数量。   
使用`torch.no_grad()`上下文管理器可以禁用梯度计算，这可以减少内存消耗并加速计算。  
遍历测试数据加载器，每次迭代都会返回一个数据批次和对应的标签,并将数据和目标移动到GPU上。  
通过模型进行前向传播，得到预测输出。  
使用损失函数计算预测输出和目标之间的损失，并将损失值累加到test_loss中。注意，这里使用了.item()方法来获取损失值的标量表示。
使用`argmax`方法获取预测类别的索引（即输出最大值所在的维度）。然后，使用`eq`方法比较预测和目标是否相等，得到一个布尔张量。最后，使用`sum`方法计算正确预测的数量，并使用`.item()`方法获取标量表示。  
最后，将累加的测试损失除以测试数据集的总大小，得到平均测试损失，并打印测试集的平均损失和准确率。

In [9]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim = 1)
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}({100. * correct / len(test_loader.dataset):.2f}%)\n")

In [12]:
num_epochs = 30
for epoch in range(0, num_epochs + 1):
    train(epoch)
    test()

torch.save(model.state_dict(), args["model"])
torch.save(model,'model.pth')

Train Epoch: 0 [0/39209] Loss: 0.003247
Train Epoch: 0 [6400/39209] Loss: 0.096513
Train Epoch: 0 [12800/39209] Loss: 0.000312
Train Epoch: 0 [19200/39209] Loss: 0.027977
Train Epoch: 0 [25600/39209] Loss: 0.042443
Train Epoch: 0 [32000/39209] Loss: 0.001552
Train Epoch: 0 [38400/39209] Loss: 0.037046

Test set: Average loss: 0.0150, Accuracy: 10376/12630(82.15%)

Train Epoch: 1 [0/39209] Loss: 0.054251
Train Epoch: 1 [6400/39209] Loss: 0.026209
Train Epoch: 1 [12800/39209] Loss: 0.022493
Train Epoch: 1 [19200/39209] Loss: 0.010693
Train Epoch: 1 [25600/39209] Loss: 0.034440
Train Epoch: 1 [32000/39209] Loss: 0.001128
Train Epoch: 1 [38400/39209] Loss: 0.012383

Test set: Average loss: 0.0067, Accuracy: 11506/12630(91.10%)

Train Epoch: 2 [0/39209] Loss: 0.003479
Train Epoch: 2 [6400/39209] Loss: 0.000923
Train Epoch: 2 [12800/39209] Loss: 0.001812
Train Epoch: 2 [19200/39209] Loss: 0.016910
Train Epoch: 2 [25600/39209] Loss: 0.002193
Train Epoch: 2 [32000/39209] Loss: 0.002609
Train E

#### (3)打印分类报告

* 将模型设置为评估模式，并禁用梯度计算。  
* 遍历测试数据加载器`test_loader`。按批次加载测试数据集。每个批次包含一对data和target。并将数据和目标移动到GPU上。  
* 模型对输入数据进行前向传播，得到模型的输出`output`  。
* 使用`argmax(dim=1)`方法沿着类别维度找到最大值的索引，即模型预测的类别。然后使用`.cpu()`将结果移动到CPU上。最后，使用`.numpy()`将结果转换为NumPy数组，以便与`sklearn`的`classification_report`函数兼容。  
* 将当前批次的预测结果`preds`和真实标签`target`分别添加到`test_preds`和`test_targets`列表中,用于存储所有测试样本的预测结果和真实标签.  
* 打印出模型的分类报告，以便根据报告在后续改进模型或训练过程。

In [25]:
test_preds = []
test_targets = []
model.eval()
with torch.no_grad():
    for data, target in test_loader:
        data = data.to(device)
        output = model(data)
        preds = output.argmax(dim=1).cpu().numpy()
        test_preds.extend(preds)
        test_targets.extend(target.numpy())

print(classification_report(test_targets, test_preds))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        60
           1       0.98      0.98      0.98       720
           2       0.94      0.99      0.96       750
           3       0.98      0.95      0.96       450
           4       0.99      0.97      0.98       660
           5       0.94      0.94      0.94       630
           6       1.00      0.84      0.91       150
           7       0.98      0.96      0.97       450
           8       0.91      0.99      0.95       450
           9       0.99      0.98      0.98       480
          10       0.99      0.98      0.98       660
          11       0.91      0.95      0.93       420
          12       1.00      0.99      0.99       690
          13       0.99      1.00      1.00       720
          14       0.99      1.00      1.00       270
          15       0.99      0.98      0.98       210
          16       1.00      1.00      1.00       150
          17       1.00    

### 5.使用模型进行预测

#### (1)定义**被预测图片**的路径与**预测结果**的输出路径

In [12]:
pred_args = {"model":"output/trafficsignnet.pth",
        "images":"test1/test",
        "pred":"output/preds/"}

* 引入训练好的模型，打开一个包含交通标志名称CSV文件，读取所有内容，去除首尾空白字符，然后按行分割成列表。
* `split("\n")[1:]`跳过第一行，只保留了包含数据的行。
* `[l.split(",")[1] for l in labelNames]`对每一行进行分割,以逗号为分隔符提取第二列的内容，因为第二列为各交通标志的名称。最后输出结果

In [13]:
warnings.filterwarnings("ignore", category=FutureWarning)#忽略了一个FutureWarning，不影响最终的输出结果
model = TrafficSignNet(width=32, height=32, 
                       depth = 3, classes = len(np.unique(train_dataset.labels)))
model.load_state_dict(torch.load(pred_args["model"]))

labelNames = open("./archive1/archive1/signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]
print(labelNames)

['Speed limit (20km/h)', 'Speed limit (30km/h)', 'Speed limit (50km/h)', 'Speed limit (60km/h)', 'Speed limit (70km/h)', 'Speed limit (80km/h)', 'End of speed limit (80km/h)', 'Speed limit (100km/h)', 'Speed limit (120km/h)', 'No passing', 'No passing for vehicles over 3.5 metric tons', 'Right-of-way at the next intersection', 'Priority road', 'Yield', 'Stop', 'No vehicles', 'Vehicles over 3.5 metric tons prohibited', 'No entry', 'General caution', 'Dangerous curve to the left', 'Dangerous curve to the right', 'Double curve', 'Bumpy road', 'Slippery road', 'Road narrows on the right', 'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Beware of ice/snow', 'Wild animals crossing', 'End of all speed and passing limits', 'Turn right ahead', 'Turn left ahead', 'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left', 'Roundabout mandatory', 'End of no passing', 'End of no passing by vehicles over 3.5 metric']


#### (2)对给定图像进行推理

* 使用`imutils.paths`中的`list_images`方法列出指定目录下的所有图像文件路径,随后打乱图像路径列表的顺序。
* 使用`io.imread`读取图像文件，并进行一系列处理。
* 使用`OpenCV`再次读取图像,并使用`cv2.putText`方法在图像上添加文本标签，显示预测的类别。
* 使用`os.path.sep.join`构建保存预测图像的路径，`cv2.imwrite`保存图像到指定路径(./output/preds/)，统一以png格式保存。
* 处理完成后输出结果

In [21]:
imagePaths = list(paths.list_images(pred_args["images"]))
random.shuffle(imagePaths)

for i, imagePath in enumerate(imagePaths):
    image = io.imread(imagePath)
    image = transform.resize(image, (32, 32))
    image = exposure.equalize_adapthist(image, clip_limit=0.1)
    image = image.astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))
    image = torch.from_numpy(image)
    image = image.unsqueeze(0)

    model.eval()
    with torch.no_grad():
        preds = model(image)
    
    j = np.argmax(preds.numpy(), axis=1)[0]
    label = labelNames[j]

    image = cv2.imread(imagePath)
    image = cv2.resize(image, (128, 128))
    cv2.putText(image, label, (5, 15), cv2.FONT_HERSHEY_SIMPLEX,
                0.45, (0, 0, 255), 1)
    
    p = os.path.sep.join(["output/preds/", "{}.png".format(i)])
    cv2.imwrite(p, image)
print(" predicts done！")

 predicts done！


### 五、项目价值

基于GTSRB数据集进行深度学习实现交通标志识别，对于提升道路交通安全性、推动智能交通系统发展具有深远意义。它能够高效准确地识别各类交通标志，为驾驶员提供及时准确的道路信息，有效减少因误解或忽视交通标志而导致的交通事故。同时，该技术也是自动驾驶技术的重要组成部分，为实现车辆自主导航、智能避障等功能提供了有力支持，有助于构建更加安全、高效的未来交通体系。

### 六、遇到问题及解决方法

### 问题1：模型准确率不高，训练时间长

* 解决方法：对模型重新进行训练，增加训练轮次，或更换优化器。

### 问题2：引入的库较多，使用不熟练

* 解决方法：在网上查询资料及文献，学习相应的使用方法

## 七、收获

在使用PyTorch处理德国交通标志识别基准（GTSRB）数据集进行交通标志识别的过程中，使我们深刻体会到了深度学习在图像分类任务中的强大能力。通过数据预处理、模型构建、训练与调优等一系列步骤，我不仅掌握了PyTorch框架的基本操作和高级特性，还学会了如何有效地处理大规模图像数据集。此外，通过调整模型架构、优化器选择以及学习率调度策略，我深刻理解了这些超参数对模型性能的影响。更重要的是，通过解决过拟合、类别不平衡等挑战，我们学会了如何运用正则化方法、数据增强技术和损失函数调整等手段来提升模型的泛化能力。整个项目不仅锻炼了编程实践能力，还加深了对深度学习原理的理解，为我们在计算机视觉领域的进一步探索奠定了坚实的基础。